<a href="https://colab.research.google.com/github/franconoronha/treinamento-h2ia/blob/main/Semana_2_BuscasSemInformacao.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# O Problema
Sliding Puzzle - Bloco Deslizante

In [ ]:
# !wget -qq https://miro.medium.com/max/700/1*W7jg4GmEjGBypd9WPktasQ.gif
from IPython.display import Image
Image(url='https://miro.medium.com/max/700/1*W7jg4GmEjGBypd9WPktasQ.gif',width=200)

# Resolver o quebra-cabeças usando Buscas

Acabei por não utilizar o numpy, na minha cabeça fazia mais sentido representar os estados utilizando strings, já que pelos meus testes ocuparia menos memória. Porém não sei se vale a pena, visto que os arrays do numpy foram feitos para ter boa perfomance, e as strings do python são imutáveis e por isso dificultam um pouquinho a manipulação.

Comparação de memória:

In [12]:
import numpy as np
from sys import getsizeof
string = "123456789"
np_array = np.array([[1,2,3],[4,5,6],[7,8,9]])

print("Tamanho de um string de 9 char: {} bytes".format(getsizeof(string)))
print("Tamanho de uma matriz numpy 3x3: {} bytes".format(getsizeof(np_array)))

Tamanho de um string de 9 char: 58 bytes
Tamanho de uma matriz numpy 3x3: 192 bytes


Importação de bibliotecas, definição de funções utilizadas e geração de um puzzle "aleatório".

In [19]:
from collections import deque
from random import seed, randint

#seed(321)

class Tree:
    def __init__(self, data, parent):
        self.children = []
        self.data = data
        self.parent = parent

""" Representando o puzzle como uma string
* representa o espaço vazio
Movimentos possíveis
(considerando que quem se move é o * e ele troca de lugar com a peça pra onde tá indo)
- pra cima: -3 posições
- pra baixo: +3 
- pra direita: +1
- pra esquerda: -1

1.só é possível se (pos_atual + movimento) for maior ou igual a 0 e menor que len(puzzle)  
2.não é possível mexer para esquerda na coluna da esquerda (posições 0, 3, 6 % 3 == 0)
3.não é possível mexer para direita na coluna da direita (posições 2, 5, 8 % 3 == 2)
"""

def gerar_puzzle():
  puzzle = ""
  numeros = ["1", "2", "3", "4", "5", "6", "7", "8"]
  for x in range(0, len(numeros)):
    char = numeros.pop(randint(0, len(numeros) - 1))
    puzzle += char
  puzzle += '*'
  return puzzle

def get_movimentos_legais(puzzle):
  ind = puzzle.find("*")
  moves = []
  for x in [3, -3, 1, -1]:
    if x == -1 and ind % 3 == 0:
      continue
    elif x == 1 and ind % 3 == 2:
      continue
    if((ind + x) < len(puzzle) and (ind + x) >= 0):
      moves.append(x)
  return moves

def swap(puzzle, move):
  index = puzzle.find('*')
  move += index
  p = list(puzzle)
  p[index], p[move] = p[move], p[index]
  return ''.join(p)

solucao = "12345678*"
puzzle = gerar_puzzle()
print("Puzzle gerado: {}".format(puzzle))

Puzzle gerado: 51326784*


## Busca em largura

In [20]:
def busca_largura(puzzle):
  global solucao
  borda = deque()
  visitados = set()
  contador = 0

  arvore = Tree(puzzle, None) # arvore de busca
  visitados.add(puzzle)       # estados gerados/"visitados"
  borda.append(arvore)

  while len(borda) > 0:
      atual = borda.popleft()
      if atual.data == solucao:
          break
      moves = get_movimentos_legais(atual.data) 
      novos_estados = [swap(atual.data, x) for x in moves]

      # Poderia pegar os estados diretos porém ficou bem confuso
      #ind = atual.data.find("*")
      #novos_estados = [swap(atual.data, x) for x in [y for y in [3, -3 , 1, -1] if (ind + y) < len(atual.data) and (ind + y) >= 0]]
      for estado in novos_estados:
          if estado not in visitados:
              contador += 1
              novo_nodo = Tree(estado, atual)
              #arvore.children.append(novo_nodo) 
              borda.append(novo_nodo)
              visitados.add(estado) 

  if atual.data == solucao:
      print("Solucionado.")
      #while atual != None:
      #    print(atual.data)
      #    atual = atual.parent
  else:
      print("Não existe uma solução")
  print("Estados gerados: {}".format(contador))
  print("Memoria: {} bytes".format(getsizeof(arvore) + getsizeof(visitados)))

busca_largura(puzzle)

Solucionado.
Estados gerados: 32625
Memoria: 2097448 bytes


## Busca em Profundidade


Utilizei a mesma função, só que implementei uma pilha com a double ended queue. Infelizmente ainda não consegui apagar os caminhos fechados, que seria a vantagem de se utilizar a busca em profundidade. Não sei se foi porque eu fiz de forma iterativa.

In [21]:
def busca_profundidade(puzzle):
  global solucao
  contador = 0
  borda = deque()
  visitados = set()

  arvore = Tree(puzzle, None) # arvore de busca
  visitados.add(puzzle)       # estados gerados/"visitados"
  borda.append(arvore)

  while len(borda) > 0:
      atual = borda.pop()     # pop ao inves de popleft
      if atual.data == solucao:
          break
      moves = get_movimentos_legais(atual.data) 
      novos_estados = [swap(atual.data, x) for x in moves]
      for estado in novos_estados:
          if estado not in visitados:
              contador += 1
              novo_nodo = Tree(estado, atual)
              borda.append(novo_nodo)
              visitados.add(estado)  

  if atual.data == solucao:
      print("Solucionado.")
  else:
      print("Não existe uma solução")
      
  print("Estados gerados: {}".format(contador))
  print("Memoria: {} bytes".format(getsizeof(arvore) + getsizeof(visitados)))

busca_profundidade(puzzle)

Solucionado.
Estados gerados: 103972
Memoria: 4194600 bytes


## Discorra sobre o desempenho dos métodos em questões de:


1.   Consumo de memória
2.   Processamento

